**Import libraries**

In [1]:
import math
import os

import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

**Default parameters**

In [2]:
MAX_SIZE = 783
NUM_WORDS = 1000
EMBEDDING_DIM = 16
EPOCHS = 10
BATCH_SIZE = 512
OOV = 0
SARCASM_TRAINING_SIZE = 20000

**Creating DataFrames**

In [3]:
goodreads_train = pd.read_csv("kaggle/input/goodreads-books-reviews-290312/goodreads_train.csv")
goodreads_test = pd.read_csv("kaggle/input/goodreads-books-reviews-290312/goodreads_test.csv")

In [4]:
goodreads_train.sample(5)

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
256416,41c230906b7890253332054b5aaf88a4,24106033,7cbba2da70c392a46a24cbca6560b045,2,"Very slow paced, full with irrelevant informat...",Tue Nov 17 19:42:30 -0800 2015,Wed Sep 07 14:57:15 -0700 2016,Wed Sep 07 14:57:15 -0700 2016,Tue Nov 17 00:00:00 -0800 2015,0,0
663792,75f9d7bf667d882fe1e6ebb322643cd3,6402666,8ed40cb6135a15e340ba5d1cb3f006d4,4,The ending...what the hell was that??!! \n I r...,Fri Feb 19 19:20:34 -0800 2010,Wed Aug 31 08:29:25 -0700 2011,Wed Aug 31 00:00:00 -0700 2011,Sat Aug 27 00:00:00 -0700 2011,0,6
855602,422e76592e2717d5d59465d22d74d47c,18073891,b12463c01ba473e8e8a9417e5fd229ef,5,Slick's review posted at Guilty Pleasures Book...,Tue Nov 29 09:13:39 -0800 2016,Tue Nov 29 09:17:15 -0800 2016,Tue Nov 29 10:07:39 -0800 2016,NaN,3,0
766064,0be561029a4c4cbf72b42f4180bb359d,16151178,7d201d9036461fceef650a715e55a318,5,Gorgeous. Absolutely Gorgeous,Wed Jan 07 14:21:35 -0800 2015,Mon May 16 15:51:29 -0700 2016,Mon May 16 15:51:29 -0700 2016,Mon May 16 00:00:00 -0700 2016,0,0
535387,d4c49a147321e322b1b05fdadd76fe08,20557510,05faf1e9104326acd923331238bdcce9,5,"""You don't need anyone to show you how to be g...",Mon May 19 08:14:16 -0700 2014,Wed Jul 09 19:43:04 -0700 2014,Wed Jul 09 00:00:00 -0700 2014,Mon Jul 07 00:00:00 -0700 2014,14,11


In [5]:
goodreads_test.sample(5)

,user_id,book_id,review_id,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
305260,61d47f7eb4c29a1aea39ebe3451c80ec,33152795,a2a5b61b9d74dd618f683a75b4c37932,4.5 stars \n UM WOW \n (changed to 5 stars oka...,Sun Sep 10 18:52:16 -0700 2017,Wed Sep 27 12:13:05 -0700 2017,Fri Sep 22 00:00:00 -0700 2017,Sat Sep 16 00:00:00 -0700 2017,1,0
78259,d174e7b353f97cc14ff7347b34b23164,28954137,db024eac169025fed3c6a34f2244587c,Welcome to the Shadowhunter Academy: 4/5 \n Th...,Sat Mar 05 07:31:30 -0800 2016,Wed Sep 27 00:14:03 -0700 2017,Sun Mar 27 05:55:52 -0700 2016,Sat Mar 05 00:00:00 -0800 2016,4,0
357783,631e78d6f21d8a29addb05a1419f181d,5308,9c87279e747c9dc934e880ec1b8163bf,A retelling of an older Mexican folk tale and ...,Wed Feb 29 18:22:20 -0800 2012,Sat Jan 24 05:29:18 -0800 2015,Thu Mar 08 20:04:57 -0800 2012,Wed Feb 29 00:00:00 -0800 2012,0,0
140363,e477d673db66fd4524ca65a88e6bcac2,23569787,ca7dde88e3de4a09cb7fe0b3075dacc4,This is long and complicated and detailed and ...,Wed Mar 30 22:43:57 -0700 2016,Fri Aug 18 18:50:35 -0700 2017,Sun Apr 03 00:07:18 -0700 2016,Wed Mar 30 00:00:00 -0700 2016,0,0
463650,c60cc2ac07e6adc15bfafe1129db0d64,16096824,c446e88ce683ae6449e5b8580ea2714b,Update May 2017: After a re-read I don't think...,Sat Mar 14 00:04:09 -0700 2015,Mon Jun 05 05:43:40 -0700 2017,Wed May 24 19:29:51 -0700 2017,Mon May 15 00:00:00 -0700 2017,4,0


**Cleaning Data**

In [6]:
train_df = goodreads_train.drop(columns=['user_id', 'book_id', 'date_added', 'date_updated', 'read_at', 'started_at'],
                                axis=0)
train_df.head()

,review_id,rating,review_text,n_votes,n_comments
0,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,This is a special book. It started slow for ab...,28,1
1,a5d2c3628987712d0e05c4f90798eb67,3,Recommended by Don Katz. Avail for free in Dec...,1,0
2,2ede853b14dc4583f96cf5d120af636f,3,"A fun, fast paced science fiction thriller. I ...",22,0
3,ced5675e55cd9d38a524743f5c40996e,0,Recommended reading to understand what is goin...,5,1
4,332732725863131279a8e345b63ac33e,4,"I really enjoyed this book, and there is a lot...",9,1


In [7]:
train_df.dtypes

review_id      object
rating          int64
review_text    object
n_votes         int64
n_comments      int64
dtype: object

In [8]:
y_train = train_df['rating']
x_train = train_df.drop('rating', axis=1)

In [9]:
x_train.head()

,review_id,review_text,n_votes,n_comments
0,dfdbb7b0eb5a7e4c26d59a937e2e5feb,This is a special book. It started slow for ab...,28,1
1,a5d2c3628987712d0e05c4f90798eb67,Recommended by Don Katz. Avail for free in Dec...,1,0
2,2ede853b14dc4583f96cf5d120af636f,"A fun, fast paced science fiction thriller. I ...",22,0
3,ced5675e55cd9d38a524743f5c40996e,Recommended reading to understand what is goin...,5,1
4,332732725863131279a8e345b63ac33e,"I really enjoyed this book, and there is a lot...",9,1


In [10]:
y_train.head()

0    5
1    3
2    3
3    0
4    4
Name: rating, dtype: int64

In [11]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

In [12]:
x_train['review_id'] = le.fit_transform(x_train['review_id'])

In [13]:
x_train.head()

,review_id,review_text,n_votes,n_comments
0,786842,This is a special book. It started slow for ab...,28,1
1,583423,Recommended by Don Katz. Avail for free in Dec...,1,0
2,165147,"A fun, fast paced science fiction thriller. I ...",22,0
3,727692,Recommended reading to understand what is goin...,5,1
4,179941,"I really enjoyed this book, and there is a lot...",9,1


**NLP**

In [14]:
tokenizer = Tokenizer(num_words=NUM_WORDS, oov_token=OOV)

In [15]:
def get_sequences(tokenizer, review):
    sequences = tokenizer.texts_to_sequences(review)
    padded_sequences = pad_sequences(sequences, truncating='post', maxlen=MAX_SIZE, padding='post')
    return padded_sequences

In [16]:
def tokenizer_func(data_rating, data_review):
    tokenizer.fit_on_texts(data_review)

    train_labels = data_rating.iloc[math.floor(int(len(data_rating) / 8)):]
    train_examples = data_review.iloc[math.floor(int(len(data_review) / 8)):]
    test_examples = data_review.iloc[:math.floor(int(len(data_review) / 8))]
    test_labels = data_rating.iloc[:math.floor(int(len(data_rating) / 8))]

    padded_train = get_sequences(tokenizer, train_examples)
    padded_test = get_sequences(tokenizer, test_examples)

    return np.array(padded_train), np.array(padded_test), np.array(train_labels), np.array(test_labels)

In [17]:
padded_train, padded_test, train_labels, test_labels = tokenizer_func(y_train, x_train['review_text'])

In [18]:
padded_train[0]

array([  4,  12, 294,  16,   6,   1, 566,  31,   3,   9,  10, 456,  42,
         4, 151,  88, 597,   1,  59,  10,   6,   1,   7,   6,   1,   1,
        21,  10, 635,  16,   2,   1,   1, 459,  45,  34, 185,   1,  27,
         6, 428,   5,   1,   2, 223,   7,   2, 888,   1,   1, 388,  42,
        80,   4, 134,  47,   1, 362,   3, 566,  32,  42,   4,  12, 294,
        16,   4,  91,  37, 291,  13,  14,  15,   9, 167, 209, 124, 207,
        25, 132,  48, 156, 290,   3,  84,   1, 255, 111, 383, 242, 124,
         3,  42, 180,   7, 460,  10,   1,   1,  24,   2, 434,  70, 914,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

**Sarcasm detection**

In [19]:
data = pd.read_json('kaggle/input/sarcasmjson/sarcasm.json', lines=True)

In [20]:
# Iterating through the json data and loading the requisite values into our python lists
sentences = data['headline']
labels = data['is_sarcastic']

In [21]:
training_sentences = sentences[0:SARCASM_TRAINING_SIZE]
testing_sentences = sentences[SARCASM_TRAINING_SIZE:]

In [22]:
training_labels = labels[0:SARCASM_TRAINING_SIZE]
testing_labels = labels[SARCASM_TRAINING_SIZE:]

In [23]:
tokenizer.fit_on_texts(training_sentences)

In [24]:
# Creating training sequences and padding them
training_padded = get_sequences(tokenizer, training_sentences)
testing_padded = get_sequences(tokenizer, testing_sentences)

In [25]:
# Converting all variables to numpy arrays, to be able to work with tf version 2
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [26]:
# Creating a model for sentiment analysis
sarcasm_model = tf.keras.Sequential([
    # Adding an Embedding layer for Neural Network to learn the vectors
    tf.keras.layers.Embedding(NUM_WORDS, EMBEDDING_DIM, input_length=MAX_SIZE),
    # Global Average pooling is similar to adding up vectors in this case
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [27]:
sarcasm_model.compile(loss='binary_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])

In [28]:
sarcasm_model.fit(training_padded, training_labels, epochs=EPOCHS,
                    validation_data=(testing_padded, testing_labels))

Epoch 1/10
625/625 [==============================] - 6s 6ms/step - loss: 0.6864 - accuracy: 0.5585 - val_loss: 0.6847 - val_accuracy: 0.5633
Epoch 2/10
625/625 [==============================] - 4s 6ms/step - loss: 0.6847 - accuracy: 0.5603 - val_loss: 0.6835 - val_accuracy: 0.5633
Epoch 3/10
625/625 [==============================] - 4s 6ms/step - loss: 0.6807 - accuracy: 0.5603 - val_loss: 0.6742 - val_accuracy: 0.5633
Epoch 4/10
625/625 [==============================] - 3s 5ms/step - loss: 0.6571 - accuracy: 0.5738 - val_loss: 0.6328 - val_accuracy: 0.5840
Epoch 5/10
625/625 [==============================] - 3s 5ms/step - loss: 0.5999 - accuracy: 0.6775 - val_loss: 0.5869 - val_accuracy: 0.6369
Epoch 6/10
625/625 [==============================] - 3s 5ms/step - loss: 0.5433 - accuracy: 0.7300 - val_loss: 0.5192 - val_accuracy: 0.7450
Epoch 7/10
625/625 [==============================] - 3s 5ms/step - loss: 0.5016 - accuracy: 0.7557 - val_loss: 0.4986 - val_accuracy: 0.7511
Epoch 

In [29]:
sarcasm_prediction_train = sarcasm_model.predict(padded_train)
sarcasm_prediction_test = sarcasm_model.predict(padded_test)

3516/3516 [==============================] - 4s 1ms/step


**Reshaping data**


In [30]:
sarcasm_prediction_train.shape

(787500, 1)

In [31]:
padded_train.shape

(787500, 783)

In [32]:
padded_train = np.concatenate((padded_train, np.array(sarcasm_prediction_train.flatten())[:, None]), axis=1)
padded_test = np.concatenate((padded_test, np.array(sarcasm_prediction_test.flatten())[:, None]), axis=1)

In [33]:
padded_train.shape

(787500, 784)

In [34]:
padded_train = np.reshape(padded_train, (1 - math.floor(len(y_train) / 8), 28, 28))
padded_test = np.reshape(padded_test, (math.floor(int(len(x_train['review_text']) / 8)), 28, 28))

In [35]:
padded_train[0]

array([[4.0000000e+00, 1.2000000e+01, 2.9400000e+02, 1.6000000e+01,
        6.0000000e+00, 1.0000000e+00, 5.6600000e+02, 3.1000000e+01,
        3.0000000e+00, 9.0000000e+00, 1.0000000e+01, 4.5600000e+02,
        4.2000000e+01, 4.0000000e+00, 1.5100000e+02, 8.8000000e+01,
        5.9700000e+02, 1.0000000e+00, 5.9000000e+01, 1.0000000e+01,
        6.0000000e+00, 1.0000000e+00, 7.0000000e+00, 6.0000000e+00,
        1.0000000e+00, 1.0000000e+00, 2.1000000e+01, 1.0000000e+01],
       [6.3500000e+02, 1.6000000e+01, 2.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 4.5900000e+02, 4.5000000e+01, 3.4000000e+01,
        1.8500000e+02, 1.0000000e+00, 2.7000000e+01, 6.0000000e+00,
        4.2800000e+02, 5.0000000e+00, 1.0000000e+00, 2.0000000e+00,
        2.2300000e+02, 7.0000000e+00, 2.0000000e+00, 8.8800000e+02,
        1.0000000e+00, 1.0000000e+00, 3.8800000e+02, 4.2000000e+01,
        8.0000000e+01, 4.0000000e+00, 1.3400000e+02, 4.7000000e+01],
       [1.0000000e+00, 3.6200000e+02, 3.000000

**CNN Training**

In [36]:
cnn_model = tf.keras.models.Sequential()

In [37]:
cnn_model.add(tf.keras.layers.Conv2D(8, (3, 3), activation=tf.keras.activations.tanh, padding='same'))
cnn_model.add(tf.keras.layers.Conv2D(8, (3, 3), activation=tf.keras.activations.tanh, padding='same'))
cnn_model.add(tf.keras.layers.MaxPool2D())

cnn_model.add(tf.keras.layers.Conv2D(16, (3, 3), activation=tf.keras.activations.tanh, padding='same'))
cnn_model.add(tf.keras.layers.Conv2D(16, (3, 3), activation=tf.keras.activations.tanh, padding='same'))
cnn_model.add(tf.keras.layers.MaxPool2D())

cnn_model.add(tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.tanh, padding='same'))
cnn_model.add(tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.tanh, padding='same'))
cnn_model.add(tf.keras.layers.MaxPool2D())

cnn_model.add(tf.keras.layers.Flatten())
cnn_model.add(tf.keras.layers.Dense(32, activation=tf.keras.activations.relu))  # tf.keras.activations.tanh
cnn_model.add(tf.keras.layers.Dense(16, activation=tf.keras.activations.relu))  # tf.keras.activations.tanh
cnn_model.add(tf.keras.layers.Dense(6,
                                    activation=tf.keras.activations.softmax))  # model.add(tf.keras.layers.Dense(1, activation=tf.keras.activations.softmax))

In [38]:
cnn_model.compile(optimizer=tf.keras.optimizers.SGD(0.1, momentum=0.9),
                  loss=tf.keras.losses.categorical_crossentropy,
                  metrics=[tf.keras.metrics.categorical_accuracy])

In [39]:
cnn_model.fit(padded_train,
              train_labels,
              batch_size=BATCH_SIZE,
              epochs=EPOCHS
              )

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

**Testing model**

In [ ]:
padded_train = padded_train / MAX_SIZE
padded_test = padded_test / MAX_SIZE

train_labels = tf.keras.utils.to_categorical(train_labels, 6)
test_labels = tf.keras.utils.to_categorical(test_labels, 6)

padded_train = np.expand_dims(padded_train, -1)
padded_test = np.expand_dims(padded_test, -1)

cnn_predict = cnn_model.predict(padded_train)

In [ ]:
cnn_model.summary()

**Submission**

In [ ]:
sample_submission = pd.read_csv("kaggle/input/goodreads-books-reviews-290312/goodreads_sample_submission.csv")
predictions = np.argmax(cnn_predict, axis=1)
sample_submission.rating = predictions
sample_submission.to_csv("kaggle/working/submission.csv", index=False)